# Google Gen AI SDK Experimentation
***

## SDK Documentation

**GitHub**: https://github.com/google-gemini/generative-ai-python


**Official Docs**: https://ai.google.dev/gemini-api/docs

## Setup

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file in the current directory
load_dotenv()

True

In [2]:
from google import genai

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

## Text generation

https://ai.google.dev/gemini-api/docs/text-generation?lang=python

In [3]:
response = client.models.generate_content(
    model='gemini-2.0-flash', contents='How does RLHF work? Keep it short.'
)
print(response.text)

RLHF (Reinforcement Learning from Human Feedback) fine-tunes language models using human preferences.

1.  **Collect Data:** Humans rank or rate different model outputs for the same prompt.
2.  **Reward Model:** Train a model to predict human preferences based on the ranked data. This becomes the "reward function".
3.  **Reinforcement Learning:** Use reinforcement learning (e.g., PPO) to further train the language model. The reward model guides the language model to generate outputs that humans would prefer.



## Vision

https://ai.google.dev/gemini-api/docs/vision?lang=python

### Inline Data Calls

Working with local images using [Pillow](https://pypi.org/project/pillow/)

In [4]:
from google import genai
from google.genai import types
import PIL.Image



image = PIL.Image.open('./data/images/beef-burger.png')

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["What is this image?", image])

print(response.text)

The image shows a burger with fries on a wooden board. The burger is topped with lettuce, tomato, red onion, cheese and a sesame seed bun. The fries are thick cut and look crispy.



**Base64 encoded images**

You can upload public image URLs by encoding them as Base64 payloads. The following code example shows how to do this using only standard library tools:

In [5]:
from google import genai
from google.genai import types
import requests

image_path = "https://goo.gle/instrument-img"
image = requests.get(image_path)

response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=["What is this image?",
              types.Part.from_bytes(data=image.content, mime_type="image/jpeg")])

print(response.text)

The image is of a church organ console. It shows the keyboards (manuals), stops, pedals, and other controls used to play the organ.



**Working with multiple images**

To prompt with multiple images, you can provide multiple images in the call to `generate_content`. These can be in any supported format, including base64 or PIL.

In [15]:
from google import genai
from google.genai import types

import pathlib
import PIL.Image

image_path_1 = "./data/images/beef-burger.png"
image_path_2 = "./data/images/chicken-strips.jpeg" # Replace with the actual path to your second image

image_url_1 = "https://goo.gle/instrument-img" # Replace with the actual URL to your third image

pil_image = PIL.Image.open(image_path_1)

b64_image = types.Part.from_bytes(
    data=pathlib.Path(image_path_2).read_bytes(),
    mime_type="image/jpeg"
)

downloaded_image = requests.get(image_url_1)

text_prompt = (
    "What is in each image, give me a list. Which of them contains food?"
    "Make sure to talk about all 3 images I sent."
)
response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=[text_prompt,
              pil_image, b64_image, downloaded_image])

print(response.text)

Alright! Let's break down the content of each image:

**Image 1:**

*   **Food:** A burger with a sesame seed bun, lettuce, red onion, tomato, cheese and a burger patty is on a wooden board along with a side of fries.
*   **Objects:** The burger and fries are placed on a wooden cutting board which is sitting on top of a wooden table with a wooden background

**Image 2:**

*   **Food:** A paper bucket filled with fried chicken. Some fries are also visible.
*   **Objects:** A paper bucket with the contents displayed on top and a transparent background

**Image 3:**

*   **Objects:** The main focus is a complex-looking pipe organ console. The file is described as including clipping paths.
*   **Non-Food:** There is also text included within the image and surrounding it. The text mentions interesting objects, other things you may like, and includes names and URLs.

**Which images contain food?**
Images 1 and 2 both contain food items. Image 3 is not food.


Note that these inline data calls don't include many of the features available through the File API, such as getting file metadata, [listing](https://ai.google.dev/gemini-api/docs/vision?lang=python#list-files), or [deleting files](https://ai.google.dev/gemini-api/docs/vision?lang=python#delete-files).

### File API

https://ai.google.dev/gemini-api/docs/vision?lang=python#large-images

https://ai.google.dev/api/files#v1beta.media.upload

**Large image payloads**

When the combination of files and system instructions that you intend to send is larger than 20 MB in size, use the File API to upload those files.

Use the [media.upload](https://ai.google.dev/api/files#v1beta.media.upload) method of the File API to upload an image of any size.

*Note*: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but cannot be downloaded from the API. It is available at no cost in all regions where the Gemini API is available.

After uploading the file, you can make GenerateContent requests that reference the File API URI. Select the generative model and provide it with a text prompt and the uploaded image.

In [35]:
img_path = "./data/images/beef-burger.png"
file_ref = client.files.upload(file=img_path)
print(f'{file_ref=}')

response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=["What can you tell me about the ingredients contained in this image?",
              file_ref])

print(response.text)

file_ref=File(name='files/40wma83uxzqh', display_name=None, mime_type='image/png', size_bytes=12331580, create_time=datetime.datetime(2025, 3, 7, 20, 12, 27, 970356, tzinfo=TzInfo(UTC)), expiration_time=datetime.datetime(2025, 3, 9, 20, 12, 27, 950670, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 3, 7, 20, 12, 27, 970356, tzinfo=TzInfo(UTC)), sha256_hash='MjAzZmMzNDhhMjA2MjViODMwMjE2M2Y0YTQ1MzAzNDY1NmUzZGJlOGM0ZWE2YzBmNDg4MzY0M2JkNzIyNTVlYQ==', uri='https://generativelanguage.googleapis.com/v1beta/files/40wma83uxzqh', download_uri=None, state=<FileState.ACTIVE: 'ACTIVE'>, source=<FileSource.UPLOADED: 'UPLOADED'>, video_metadata=None, error=None)
Based on the image, the ingredients appear to include:

*   **Burger bun:** A sesame seed bun
*   **Beef patty:** Appears to be well-cooked and juicy
*   **Cheese:** A slice of what looks like white cheese, possibly Swiss or provolone
*   **Tomato:** A slice of fresh tomato
*   **Red onion:** Thin slices of red onion
*   **Lettuce:*

### Prompting with images

https://ai.google.dev/gemini-api/docs/vision?lang=python#prompting-images

## Count Tokens

https://ai.google.dev/gemini-api/docs/tokens?lang=python

In [8]:
response = client.models.count_tokens(
    model='gemini-2.0-flash',
    contents='The quick brown fox jumps over the lazy dog.',
)

In [9]:
response

CountTokensResponse(total_tokens=11, cached_content_token_count=None)

In [10]:
response.total_tokens

11

## System Instruction

In [24]:
sys_instruct = (
    "You are a cat. Your name is Neko."
)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=sys_instruct),
    contents=["Who are you?"]
)

In [25]:
response.text

"Meow! I'm Neko, a very important and purrfectly delightful cat. Now, where's my tuna?\n"

## Structured Output

https://ai.google.dev/gemini-api/docs/structured-output?lang=python

Using `genai.types.Schema` (can also pass equivalent `dict`)

In [16]:
# import base64
# import os
# from google import genai
# from google.genai import types


# def generate():
#     client = genai.Client(
#         api_key=os.environ.get("GEMINI_API_KEY"),
#     )

#     model = "gemini-2.0-flash"
#     contents = [
#         types.Content(
#             role="user",
#             parts=[
#                 types.Part.from_text(
#                     text="Provide a detailed recipe for a delicious pasta dish"
#                 ),
#             ],
#         ),
#     ]
#     generate_content_config = types.GenerateContentConfig(
#         temperature=1,
#         top_p=0.95,
#         top_k=40,
#         max_output_tokens=8192,
#         response_mime_type="application/json",
#         response_schema=genai.types.Schema(
#             type=genai.types.Type.OBJECT,
#             enum=[],
#             required=["summary", "instructions"],
#             properties={
#                 "summary": genai.types.Schema(
#                     type=genai.types.Type.STRING,
#                 ),
#                 "instructions": genai.types.Schema(
#                     type=genai.types.Type.ARRAY,
#                     items=genai.types.Schema(
#                         type=genai.types.Type.STRING,
#                     ),
#                 ),
#             },
#         ),
#     )

#     # Uncomment the streaming version if needed (does not work properly in a notebook environment):
#     # for chunk in client.models.generate_content_stream(
#     #     model=model,
#     #     contents=contents,
#     #     config=generate_content_config,
#     # ):
#     #     print(chunk.text, end="")

#     # Use the non-streaming API call instead:
#     response = client.models.generate_content(
#         model=model,
#         contents=contents,
#         config=generate_content_config,
#     )
#     # print(response.text)
#     return response.text

Using *Pydantic* schemas

In [17]:
from pydantic import BaseModel
from typing import List

# Example Pydantic schema for reference
# -------------------------------------
class RecipeResponse(BaseModel):
    title: str
    summary: str
    instructions: List[str]


# Example with optional field for reference
# -----------------------------------------
# from typing import Optional

# class RecipeResponse(BaseModel):
#     title: Optional[str] = None  # now optional
#     summary: str
#     instructions: List[str]


In [18]:
import base64
import os
from google import genai
from google.genai import types

def generate():
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.0-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(
                    text="Provide a detailed recipe for a delicious chicken dish"
                ),
            ],
        ),
    ]
    # Use the Pydantic schema directly in response_schema.
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        top_k=40,
        max_output_tokens=8192,
        response_mime_type="application/json",
        response_schema=RecipeResponse,
    )

    # Use the non-streaming API call.
    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
    )
    return response

In [19]:
result = generate()

In [20]:
type(result)

google.genai.types.GenerateContentResponse

`result.text` is a raw JSON string—you get exactly what the API returned as text. It's of type str and shows the full JSON output, which you might want to log or inspect directly.

In [21]:
print("Raw response text:\n")
print(result.text)
print("")
print(type(result.text))

Raw response text:

{
  "title": "Lemon Herb Roasted Chicken",
  "summary": "A flavorful and juicy whole roasted chicken infused with lemon, herbs, and garlic. Perfect for a weeknight dinner or a special occasion.",
  "instructions": [
    "Preheat oven to 425°F (220°C).",
    "Rinse a 3-4 pound whole chicken inside and out and pat dry with paper towels. Remove giblets and neck, if present.",
    "In a small bowl, combine 2 tablespoons of softened butter, 2 cloves minced garlic, zest of 1 lemon, 1 tablespoon chopped fresh rosemary, 1 tablespoon chopped fresh thyme, 1/2 teaspoon salt, and 1/4 teaspoon black pepper.",
    "Loosen the skin of the chicken breast and rub half of the herb butter mixture underneath the skin, directly onto the breast meat. Rub the remaining herb butter all over the outside of the chicken.",
    "Place the chicken in a roasting pan. Stuff the cavity with the halved lemon and the remaining herb sprigs.",
    "Tie the chicken legs together with kitchen twine.",
 

`result.parsed` is the same data, but automatically converted into a structured Python object based on the schema you provided—in this case, an instance of your RecipeResponse Pydantic model. This lets you work with the data using native Python attributes (for example, accessing parsed.summary directly) and ensures that the data conforms to your expected structure.

In [22]:
# The SDK should automatically parse the JSON into an instance of RecipeResponse.
parsed = result.parsed
print("\nParsed response as RecipeResponse instance:\n")
print(parsed)
print("")
print(type(parsed))


Parsed response as RecipeResponse instance:

title='Lemon Herb Roasted Chicken' summary='A flavorful and juicy whole roasted chicken infused with lemon, herbs, and garlic. Perfect for a weeknight dinner or a special occasion.' instructions=['Preheat oven to 425°F (220°C).', 'Rinse a 3-4 pound whole chicken inside and out and pat dry with paper towels. Remove giblets and neck, if present.', 'In a small bowl, combine 2 tablespoons of softened butter, 2 cloves minced garlic, zest of 1 lemon, 1 tablespoon chopped fresh rosemary, 1 tablespoon chopped fresh thyme, 1/2 teaspoon salt, and 1/4 teaspoon black pepper.', 'Loosen the skin of the chicken breast and rub half of the herb butter mixture underneath the skin, directly onto the breast meat. Rub the remaining herb butter all over the outside of the chicken.', 'Place the chicken in a roasting pan. Stuff the cavity with the halved lemon and the remaining herb sprigs.', 'Tie the chicken legs together with kitchen twine.', 'Roast for 1 hour a

**Note**:

Use `result.text` when you need the raw JSON (for logging or debugging) and `result.parsed` when you want the validated, typed object for further processing in your application.

If you're already using the SDK’s automatic parsing (i.e. using `result.parsed`), then the JSON has already been parsed and validated against your Pydantic schema. In that case, calling an additional validation function is usually redundant.

The function `validate_json_respons` is designed to take a raw JSON string (like what you'd get from `result.text`) and manually validate it against a provided Pydantic model. But since the SDK does that for you when you supply the schema in the configuration, there's generally no need to pass anything extra to this function.

In [23]:
from pydantic import BaseModel
from typing import List, Type, Tuple, Optional
import json


def validate_json_response(response_text: str, schema: Type[BaseModel]) -> Tuple[bool, Optional[BaseModel]]:
    """
    Validates that the given response_text is valid JSON and adheres to the provided Pydantic schema.
    
    Parameters:
        response_text (str): The raw JSON string to validate.
        schema (Type[BaseModel]): The Pydantic schema to validate against.
        
    Returns:
        Tuple[bool, Optional[BaseModel]]: A tuple where the first element is True if the JSON is valid and adheres to the schema,
                                           and the second element is the validated Pydantic model instance (or the raw data if validation fails).
    """
    try:
        data = json.loads(response_text)
    except json.JSONDecodeError as e:
        print("Invalid JSON:", e)
        return False, None

    # Print the schema details we're validating against:
    schema_json = json.dumps(schema.model_json_schema(), indent=2)
    print("Validating against schema:")
    print(schema_json)
    
    try:
        # Use model_validate (Pydantic v2) instead of parse_obj
        validated_data = schema.model_validate(data)
    except Exception as e:
        print("Schema validation failed:", e)
        return False, data

    # print("Response is valid and adheres to the schema.")
    return True, validated_data



In [24]:
# Call generate() to get the response JSON string
result = generate()

# raw JSON string
raw_response = result.text

# Validate the JSON response using the RecipeResponse schema
is_valid, validated_model = validate_json_response(raw_response, RecipeResponse)

if is_valid:
    print("Validation successful!")
    # Pretty-print the validated model using json.dumps on model_dump()
    # print(json.dumps(validated_model.model_dump(), indent=2))
else:
    print("Validation failed.")

Validating against schema:
{
  "properties": {
    "title": {
      "title": "Title",
      "type": "string"
    },
    "summary": {
      "title": "Summary",
      "type": "string"
    },
    "instructions": {
      "items": {
        "type": "string"
      },
      "title": "Instructions",
      "type": "array"
    }
  },
  "required": [
    "title",
    "summary",
    "instructions"
  ],
  "title": "RecipeResponse",
  "type": "object"
}
Validation successful!


## Function calling

https://ai.google.dev/gemini-api/docs/function-calling

### Manual

Automatic function calling is the default. Here we disable it.

In [25]:
from google import genai
from google.genai import types
client = genai.Client()

def get_current_weather(location: str) -> str:
    """Get the current whether in a given location.

    Args:
        location: required, The city and state, e.g. San Franciso, CA
        unit: celsius or fahrenheit
    """
    print(f'Called with: {location=}')
    return "23C"

response = client.models.generate_content(
   model='gemini-2.0-flash',
   contents="What is the weather like in Boston?",
   config=types.GenerateContentConfig(
       tools=[get_current_weather],
       automatic_function_calling={'disable': True},
   ),
)

function_call = response.candidates[0].content.parts[0].function_call

/Users/marabian/Notebooks/google-genai-sdk-notebooks/.venv/lib/python3.12/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [26]:
function_call

FunctionCall(id=None, args={'location': 'Boston, MA'}, name='get_current_weather')

In [27]:
# New cell to execute the function call extracted from the response
if function_call:
    print("Function call details:", function_call)
    # Extract the function name and arguments
    func_name = function_call.name
    func_args = function_call.args

    # Based on the function name, call the appropriate function with the provided arguments.
    if func_name == "get_current_weather":
        result = get_current_weather(**func_args)
        print("Function call result:", result)
    else:
        print("No matching function found for", func_name)
else:
    print("No function call detected in the response.")


Function call details: id=None args={'location': 'Boston, MA'} name='get_current_weather'
Called with: location='Boston, MA'
Function call result: 23C


### Automatic

In [28]:
from google import genai
from google.genai import types
client = genai.Client()

def get_current_weather(city: str) -> str:
    return "23C"

response = client.models.generate_content(
   model='gemini-2.0-flash',
   contents="What is the weather like in Boston?",
   config=types.GenerateContentConfig(
       tools=[get_current_weather] 
   ),
)

In [29]:
response.text

'It is 23C in Boston.\n'

In [33]:
# !pip install cowsay

# import cowsay  # Ensure you have installed cowsay via pip

# def generate_cowsay(text: str) -> str:
#     # Use cowsay.cow instead of cowsay.cowsay
#     return cowsay.cow(text)

## Code Execution

https://ai.google.dev/gemini-api/docs/code-execution?lang=python

### Basic Example

In [40]:
from google import genai
from google.genai import types

client = genai.Client()

response = client.models.generate_content(
  model='gemini-2.0-flash',
  contents='What is the sum of the first 50 prime numbers? '
           'Generate and run code for the calculation, and make sure you get all 50.',
  config=types.GenerateContentConfig(
    tools=[types.Tool(
      code_execution=types.ToolCodeExecution
    )]
  )
)

In [42]:
from IPython.display import display, Markdown, HTML, Image

def display_code_execution_result(response):
  for part in response.candidates[0].content.parts:
    if part.text is not None:
      display(Markdown(part.text))
    if part.executable_code is not None:
      code_html = f'<pre style="background-color: #BBBBEE;">{part.executable_code.code}</pre>' # Change code color
      display(HTML(code_html))
    if part.code_execution_result is not None:
      display(Markdown(part.code_execution_result.output))
    if part.inline_data is not None:
      display(Image(data=part.inline_data.data, format="png"))
    display(Markdown("---"))

display_code_execution_result(response)

Okay, I need to find the sum of the first 50 prime numbers. Here's my plan:

1.  Generate a list of prime numbers until I have 50 of them.
2.  Calculate the sum of those 50 prime numbers.

Here's the Python code to do that:



---

---

result=5117


---

The sum of the first 50 prime numbers is 5117.


---

### Interleaving Example

In [43]:
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=(
        "Please do the following in one response:\n\n"
        "Calculation 1: Write and run code that calculates the sum of the numbers from 1 to 10.\n"
        "Text Output: After the first calculation, output the text message: 'The sum of 1 through 10 has been computed successfully!'\n"
        "Calculation 2: Then, write and run code that calculates the factorial of 6.\n"
        "Make sure the response interleaves the code outputs with the text message accordingly."
    ),
    config=types.GenerateContentConfig(
        tools=[types.Tool(
            code_execution=types.ToolCodeExecution
        )]
    )
)


In [38]:
display_code_execution_result(response)

---

sum_result=55


---

The sum of 1 through 10 has been computed successfully!


---

---

factorial_result=720


---

I have now completed both calculations and interspersed the output messages as requested.


---

### Input/output (I/O)

Starting with Gemini 2.0 Flash, code execution supports file input and graph output. Using these new input and output capabilities, you can upload CSV and text files, ask questions about the files, and have Matplotlib graphs generated as part of the response.



https://ai.google.dev/gemini-api/docs/code-execution?lang=python#input-output

## Search grounding

https://ai.google.dev/gemini-api/docs/grounding?lang=python

`GoogleSearch` (Gemini>=2.0) and `GoogleSearchRetrieval` (Gemini < 2.0) are tools that allow the model to retrieve public web data for grounding, powered by Google.



In [185]:
from google import genai
from google.genai import types
client = genai.Client()

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='What is the Google stock price?',
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                google_search=types.GoogleSearch()
            )
        ]
    )
)

In [186]:
response.text

'As of March 5, 2025, at 8:19 AM UTC, the price of Alphabet Inc. (Google) Class C stock (GOOG) is approximately $172.61. It has decreased by 2.07% in the past 24 hours.\n'

In [184]:
for part in response.candidates[0].content.parts:
    print(part)

video_metadata=None thought=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=None inline_data=None text='As of March 5, 2025, the price of Alphabet Inc (Google) Class C (GOOG) is around $172.61.\nIt has decreased by approximately -2.07% in the past 24 hours.\n'


## Document Understanding

https://ai.google.dev/gemini-api/docs/document-processing?lang=python

Needs testing for `Gemini 2.0`, docs are for older versions of Gemini.

Supports:

* PDF - `application/pdf`
* JavaScript - `application/x-javascript, text/javascript`
* Python - `application/x-python, text/x-python`
* TXT - `text/plain`
* HTML - `text/html`
* CSS - `text/css`
* Markdown - `text/md`
* CSV - `text/csv`
* XML - `text/xml`
* RTF - `text/rtf`

## Async

https://github.com/google-gemini/generative-ai-python?tab=readme-ov-file#async

## Context caching



https://github.com/google-gemini/generative-ai-python?tab=readme-ov-file#context-caching

## Embed Content

Docs: https://ai.google.dev/gemini-api/docs/embeddings

Article: https://developers.googleblog.com/en/gemini-embedding-text-model-now-available-gemini-api/

In [14]:
response = client.models.embed_content(
   model='text-embedding-004',
   contents='Hello world',
)

In [15]:

response = client.models.embed_content(
   model='gemini-embedding-exp-03-07',
   contents='Hello world',
)

In [17]:
# response.embeddings

## Tune a Model

https://github.com/google-gemini/generative-ai-python?tab=readme-ov-file#tune-a-model

## File API

https://ai.google.dev/api/files

### Upload Files

In [44]:
from google import genai

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# Upload a file
poem_file = client.files.upload(file="./data/documents/poem.txt")
print(poem_file)
# Files will auto-delete after a period.
print(poem_file.expiration_time)

name='files/ai4wlw91zl5n' display_name=None mime_type='text/plain' size_bytes=123 create_time=datetime.datetime(2025, 3, 7, 20, 19, 1, 640221, tzinfo=TzInfo(UTC)) expiration_time=datetime.datetime(2025, 3, 9, 20, 19, 1, 627617, tzinfo=TzInfo(UTC)) update_time=datetime.datetime(2025, 3, 7, 20, 19, 1, 640221, tzinfo=TzInfo(UTC)) sha256_hash='Nzg3YmZmOGQyYjlkMWYzNGIxYTgxMGMyNTU1ODU1OWQ1NjdlNWJjOTkzYjdlNGYyY2VkNmJmZDMxNDE0MTIwMA==' uri='https://generativelanguage.googleapis.com/v1beta/files/ai4wlw91zl5n' download_uri=None state=<FileState.ACTIVE: 'ACTIVE'> source=<FileSource.UPLOADED: 'UPLOADED'> video_metadata=None error=None
2025-03-09 20:19:01.627617+00:00


### Get Files

In [46]:

file_name = poem_file.name
print(file_name)  # "files/*"

myfile = client.files.get(name=file_name)
print(myfile)

files/ai4wlw91zl5n
name='files/ai4wlw91zl5n' display_name=None mime_type='text/plain' size_bytes=123 create_time=datetime.datetime(2025, 3, 7, 20, 19, 1, 640221, tzinfo=TzInfo(UTC)) expiration_time=datetime.datetime(2025, 3, 9, 20, 19, 1, 627617, tzinfo=TzInfo(UTC)) update_time=datetime.datetime(2025, 3, 7, 20, 19, 1, 640221, tzinfo=TzInfo(UTC)) sha256_hash='Nzg3YmZmOGQyYjlkMWYzNGIxYTgxMGMyNTU1ODU1OWQ1NjdlNWJjOTkzYjdlNGYyY2VkNmJmZDMxNDE0MTIwMA==' uri='https://generativelanguage.googleapis.com/v1beta/files/ai4wlw91zl5n' download_uri=None state=<FileState.ACTIVE: 'ACTIVE'> source=<FileSource.UPLOADED: 'UPLOADED'> video_metadata=None error=None


### List Files

You can list all files uploaded using the File API and their URIs using [files.list](https://ai.google.dev/api/files#method:-files.list).

In [45]:
from google import genai

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

print('My files:')
for f in client.files.list():
  print(" ", f'{f.name}: {f.uri}')
  print(f"Expiration time {f.expiration_time}\n")

My files:
  files/ai4wlw91zl5n: https://generativelanguage.googleapis.com/v1beta/files/ai4wlw91zl5n
Expiration time 2025-03-09 20:19:01.627617+00:00

  files/40wma83uxzqh: https://generativelanguage.googleapis.com/v1beta/files/40wma83uxzqh
Expiration time 2025-03-09 20:12:27.950670+00:00

  files/r2cats2n293b: https://generativelanguage.googleapis.com/v1beta/files/r2cats2n293b
Expiration time 2025-03-09 19:54:19.310314+00:00

  files/x742dpxcws01: https://generativelanguage.googleapis.com/v1beta/files/x742dpxcws01
Expiration time 2025-03-09 19:53:37.662259+00:00

  files/56wvqljv3zoe: https://generativelanguage.googleapis.com/v1beta/files/56wvqljv3zoe
Expiration time 2025-03-09 19:53:17.331049+00:00



### Delete Files


In [26]:
from google import genai

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# Upload a file
poem_file = client.files.upload(file="./data/documents/poem.txt")
print(poem_file)
# Files will auto-delete after a period.
print(poem_file.expiration_time)

# Or they can be deleted explicitly.
dr = client.files.delete(name=poem_file.name)

try:
  client.models.generate_content(
      model="gemini-2.0-flash-exp",
      contents=['Finish this poem:', poem_file])
except genai.errors.ClientError as e:
  print(e.code)  # 403
  print(e.status)  # PERMISSION_DENIED
  print(e.message)  # You do not have permission to access the File .. or it may not exist.

name='files/vltb7v3922ct' display_name=None mime_type='text/plain' size_bytes=123 create_time=datetime.datetime(2025, 3, 7, 20, 7, 33, 762649, tzinfo=TzInfo(UTC)) expiration_time=datetime.datetime(2025, 3, 9, 20, 7, 33, 744718, tzinfo=TzInfo(UTC)) update_time=datetime.datetime(2025, 3, 7, 20, 7, 33, 762649, tzinfo=TzInfo(UTC)) sha256_hash='Nzg3YmZmOGQyYjlkMWYzNGIxYTgxMGMyNTU1ODU1OWQ1NjdlNWJjOTkzYjdlNGYyY2VkNmJmZDMxNDE0MTIwMA==' uri='https://generativelanguage.googleapis.com/v1beta/files/vltb7v3922ct' download_uri=None state=<FileState.ACTIVE: 'ACTIVE'> source=<FileSource.UPLOADED: 'UPLOADED'> video_metadata=None error=None
2025-03-09 20:07:33.744718+00:00
403
PERMISSION_DENIED
You do not have permission to access the File vltb7v3922ct or it may not exist.


## Prompt Engineering

### Intro to Prompting

https://ai.google.dev/gemini-api/docs/prompting-intro

### Prompting Strategies

https://ai.google.dev/gemini-api/docs/prompting-strategies

### File prompting strategies

https://ai.google.dev/gemini-api/docs/file-prompting-strategies

## Troubleshooting Guide

https://ai.google.dev/gemini-api/docs/troubleshooting?lang=python